Comparing the performance of different methods for accessing DWD TRY weather data

In [ ]:
import geopandas
import xarray as xr
import acept.dwd_try_data_handling as dh
import os
from acept.acept_constants import TRY_BAVARIAN_PATH

In [ ]:
# comb = dh.read_dwd_netcdf_file("combined_try", 2011, 1)

comb = xr.open_dataset(os.path.join(TRY_BAVARIAN_PATH, "TRY_201101.nc"))

In [ ]:
comb

In [ ]:
t = dh.read_dwd_netcdf_file("temperature", 2011, 1)
t

In [ ]:

sid = dh.read_dwd_netcdf_file("rad_direct", 2011, 1)
sid

In [ ]:
sis = dh.read_dwd_netcdf_file("rad_global", 2011, 1)
sis

In [ ]:
sid["SIS"]=sis["SIS"]
sid.info

In [ ]:
sid

In [ ]:
from acept.dwd_try_data_handling import preprocess_dwd_try_dataset
import geopandas
from shapely.geometry import Point
# centroids of buildings as geometry point
d = {'col1': ['name1'], 'geometry': [Point(11.4639, 48.2920)]}
selected_shape = geopandas.GeoDataFrame(d, crs="EPSG:4326")
selected_shape = selected_shape.to_crs(epsg=3034)
sid = preprocess_dwd_try_dataset(sid, "rad_direct")
sid.SIS.rio.write_crs("EPSG:3034", inplace=True)

Relatively expensive

In [ ]:
a = sid.rio.clip(selected_shape.geometry.values, selected_shape.crs, all_touched=True)
# Select data for the nearest latitude and longitude
a

If selected_shape or the buildings are already in the correct CRS, this is easiest:

In [ ]:
a = sid.sel(X=selected_shape.loc[0, "geometry"].x, Y=selected_shape.loc[0, "geometry"].y, method="nearest")
a

Faster

In [ ]:
import cartopy.crs as ccrs

ds_crs = ccrs.epsg(3034)

# Transform the point - src_crs is always Plate Carree for lat/lon grid
x, y = ds_crs.transform_point(11.4639, 48.2920, src_crs=ccrs.Geodetic())
print(x, y)
# Now you can select data
sid.sel(X=x, Y=y, method="nearest")

In [ ]:
a = sid.sel(X=4104500, Y=2402500, method='nearest')
a

In [ ]:
diffuse = a.SIS - a.rad_direct
diffuse

In [ ]:
diffuse_fraction = (diffuse / a.SIS).fillna(0)
diffuse_fraction

In [ ]:
a.to_pandas().loc[:,["rad_direct", "SIS"]]

Bavaria shape

In [ ]:
from acept.acept_constants import FED_STATES_PATH
import geopandas as gpd
bavaria_shape = gpd.read_file(FED_STATES_PATH)
bavaria_shape = [bavaria_shape["GEN"] == "Bayern"]